# Redes Neurais Recorrentes

### Descriçao

---
1. O objetivo deste notebook é construir uma Rede Neural Recorrente.
2. Utilizaremos o conjunto de dados proposto no [gist](https://gist.githubusercontent.com/batestin1/b8f7c0a26c9669013ba451c18e381d75/raw/e355693a14880cd8dbe2bee26a52dafdc9aaabec/gistfile1.txt

3. O problema consiste em prever as ações temporais do preço da Google na bolsa de valores
---

### Dicionário


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
Date 	  										  	  |string     | Data da alteração |
Open														  |float    | preço da abertura                        |
High		     										  |float     | preço mais alto no dia	               |
Low | float | preco mais baixo no dia
Close | float | preco de fechamento
Volume | float | Volume total do dia
  

# Instalação dos pacotes

In [ ]:
!pip install pandas numpy scikit-learn keras matplotlib

# Documentação

1. ** Pandas ** -> [Link](https://pandas.pydata.org/docs/)
2. ** Numpy ** -> [Link](https://numpy.org/doc/)
4. ** Scikit Learn ** -> [Link](https://scikit-learn.org/stable/)
5. ** Keras ** -> [Link](https://keras.io/api/)
6. ** Tensor Flow ** -> [Tensor Flow](https://www.tensorflow.org/api_docs/python/tf/keras)



# Instalando as bibliotecas

In [ ]:
import numpy as np #para manipulacao e criacao de matrizes
import matplotlib.pyplot as plt #para visualizacao dos dados
import pandas as pd #para manipulacao de dados

from sklearn.preprocessing import MinMaxScaler # para pre processamento de ml

from keras.models import Sequential # para redes neurais
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

import os # para criacao e manipulacao de pastas
from keras.models import load_model #para salvar modelos do keras


# Obtendo o dataset

In [ ]:
df = pd.read_csv('https://gist.githubusercontent.com/batestin1/b8f7c0a26c9669013ba451c18e381d75/raw/e355693a14880cd8dbe2bee26a52dafdc9aaabec/gistfile1.txt')


#### Conhecendo o dataset

In [ ]:
df.info() #visualizando as informacoes do dataset

# Pre Processamento ML

In [ ]:
# padronizando os dados com o Min e Max
training_set = df.iloc[:, 1:2].values

sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)



In [ ]:
#Estruturas de dados com 60 intervalos
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
# Ajustar Formato
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Construindo nossa RNN

In [ ]:
#Inicializando a RNA
regressor = Sequential()
#Criando as camadas
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

In [ ]:
#Inicializando a RNA
regressor = Sequential()
#Criando as camadas
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 1))

# Salvando o modelo

In [ ]:
folder = 'rnm/'

# Verifica se o diretório existe e, se não existir, cria o diretório
if not os.path.exists(folder):
    os.makedirs(folder)

# Salva o modelo no diretório especificado
regressor.save(os.path.join(folder, 'rede_neural_recorrente.h5'))

# Importando o modelo

In [ ]:
model = load_model(os.path.join(folder, 'rede_neural_recorrente.h5'))

In [ ]:
df_real = pd.read_csv('https://gist.githubusercontent.com/batestin1/876e82c27509f2963115432a10c25512/raw/cf9a54436f3936a1b2b15b7a852e0468fc5b8be6/gistfile1.txt')
real_stock_price = df_real.iloc[:, 1:2].values

In [ ]:
#Obtendo os dados previstos
dataset_total = pd.concat((df['Open'], df_real['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(df_real) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualizando o Resultado

In [ ]:
plt.plot(real_stock_price, color = 'red', label = 'Dados Reais de Ações do Google')
plt.plot(predicted_stock_price, color = 'blue', label = 'Dados Previstos de Ações do Google')
plt.title('Previsão de Preços de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preços de Ações do Google')
plt.legend()
plt.show()